en esta version tomo para el marketshare FEB19, si en algun mes de 2019 no esta le calcula el promedio de 2019 al final +  la arquitectura de German. si la prediccion es negativa pone 0. clusteriza con 15 (categorias, brand y customer)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, Bidirectional, LSTM, Dense
import joblib
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='keras')




KeyboardInterrupt: 

In [29]:
# Paso 1: Cargar y preprocesar los datos
file_path = "C:/Users/Usuario/desktop/vero2/final_dataset_completo_con_ceros.csv"
df = pd.read_csv(file_path, sep='\t')

df = df.drop('Unnamed: 0', axis=1)
df.head(12)

filtered_df = df[df['product_id'].between(20001, 20012)]

# Opcional: Ver las primeras filas del DataFrame filtrado
print(filtered_df.head())

filtered_df=df

   customer_id  product_id  periodo  plan_precios_cuidados  cust_request_qty  \
0        10001       20001   201812                    0.0              20.0   
1        10001       20001   201901                    0.0              53.0   
2        10001       20001   201902                    0.0              39.0   
3        10001       20001   201903                    0.0              23.0   
4        10001       20001   201904                    0.0              33.0   

   cust_request_tn         tn cat1         cat2     cat3  brand  sku_size  \
0        254.62373  254.62373   HC  ROPA LAVADO  Liquido  ARIEL      3000   
1        393.26092  386.60688   HC  ROPA LAVADO  Liquido  ARIEL      3000   
2        309.90610  309.90610   HC  ROPA LAVADO  Liquido  ARIEL      3000   
3        142.87158  130.54927   HC  ROPA LAVADO  Liquido  ARIEL      3000   
4        364.37071  364.37071   HC  ROPA LAVADO  Liquido  ARIEL      3000   

  descripcion quarter  month  close_quarter   age mes_in

In [30]:

# Reemplazar 082019 por promedio 07 y 09
df['periodo'] = df['periodo'].astype(str).str.strip()
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], var_name='periodo', value_name='tn')
df.set_index(['product_id', 'customer_id', 'periodo'], inplace=True)
df.update(updated_sales.set_index(['product_id', 'customer_id', 'periodo']))
df.reset_index(inplace=True)

# Aplicar LabelEncoder a las columnas categóricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Agrupar las ventas por periodo, cat1, cat2, cat3, brand y customer_id
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')
grouped_df = df.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({'tn': 'sum'}).reset_index()

# Paso 2: Calcular los ratios incluyendo customer_id
df_diciembre_2019 = df[(df['periodo'].dt.year == 2019) & (df['periodo'].dt.month == 12)]
grouped_sales_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['tn'].sum().reset_index()
group_totals_2019 = df_diciembre_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'customer_id'])['tn'].sum().reset_index()
ratios_2019 = pd.merge(grouped_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'customer_id', 'product_id'])['ratio'].to_dict()

# Paso 3: Clustering con DTW
pivot_df = grouped_df.pivot_table(index=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], columns='periodo', values='tn', fill_value=0)
pivot_df_values = pivot_df.values

# Escalar los datos
scalers = {}
scaler = StandardScaler()
pivot_df_scaled = scaler.fit_transform(pivot_df_values)


# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')




['scalers.pkl']

In [ ]:
#Determinar el número óptimo de clusters utilizando el diagrama de codo
distortions = []
K = range(6, 15)
for k in K:
    kmeans_model = TimeSeriesKMeans(n_clusters=k, metric="dtw", verbose=0, random_state=42)
    kmeans_model.fit(pivot_df_scaled)
    distortions.append(kmeans_model.inertia_)

 Visualizar el diagrama de codo
plt.figure(figsize=(10, 6))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [8]:
!pip install tqdm
from tqdm import tqdm

In [31]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, Bidirectional, LSTM, Dense
from tslearn.clustering import TimeSeriesKMeans







# Función para crear secuencias de tiempo
def crear_secuencias(data, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(data) - n_steps - step_ahead):
        X.append(data[['tn']].iloc[i:i + n_steps].values)
        y.append(data['tn'].iloc[i + n_steps + step_ahead - 1])
    return np.array(X), np.array(y)

# Función para construir el modelo LSTM
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((input_shape[0] // 2, 64)))
    model.add(Bidirectional(LSTM(50, activation='relu')))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Función para procesar los datos en lotes
def process_data_in_batches(scaled_df, clusters, n_steps, step_ahead=1, batch_size=10000):
    models = {}
    predictions = []
    scalers = {}

    # Iterar sobre cada cluster
    unique_clusters = scaled_df['cluster'].unique()

    for cluster in tqdm(unique_clusters, desc='Processing clusters'):
        group_data = scaled_df[scaled_df['cluster'] == cluster]
        group_data = group_data.sort_values(by='periodo')

        # Dividir en lotes
        num_batches = len(group_data) // batch_size + 1
        for i in range(num_batches):
            batch_data = group_data.iloc[i * batch_size:(i + 1) * batch_size]

            X, y = crear_secuencias(batch_data, n_steps, step_ahead)

            if len(X) == 0 or len(y) == 0:
                continue

            # Escalar los datos
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)

            # Construir y entrenar modelo LSTM
            model = build_lstm_model((X_scaled.shape[1], X_scaled.shape[2]))
            model.fit(X_scaled, y, epochs=100, verbose=0)

            # Guardar modelo en el diccionario
            model_key = f"{cluster}_batch_{i}"
            models[model_key] = model
            scalers[model_key] = scaler

            # Hacer predicciones para el último paso de tiempo
            X_pred = batch_data[['tn']].values[-(n_steps + step_ahead):-step_ahead].reshape((1, n_steps, 1))
            X_pred_scaled = scalers[model_key].transform(X_pred.reshape(-1, 1)).reshape(X_pred.shape)
            pred = model.predict(X_pred_scaled, verbose=0)
            pred_original_scale = scalers[model_key].inverse_transform(pred.reshape(-1, 1)).flatten()
            pred_original_scale = np.clip(pred_original_scale, 0, None)
            predictions.append([cluster, pred_original_scale[0]])

    return models, predictions

# Suponiendo que tienes pivot_df_scaled, grouped_df, ratio_dict, y df definidos previamente
optimal_k = 15

# Ajustar KMeans a los datos escalados
kmeans_model = TimeSeriesKMeans(n_clusters=optimal_k, metric="dtw", verbose=0, random_state=42)
clusters = kmeans_model.fit_predict(pivot_df_scaled)


# Añadir el cluster al DataFrame original
pivot_df['cluster'] = clusters
grouped_df = pd.merge(grouped_df, pivot_df[['cluster']], on=['cat1', 'cat2', 'cat3', 'brand', 'customer_id'], right_index=True)

# Guardar el resultado en un archivo CSV (opcional)
grouped_df.to_csv('grouped_with_clusters.csv', index=False)
# Mostrar el DataFrame final con los números de grupo
display(grouped_df)



# Escalar los datos en grouped_df
scaler = StandardScaler()
grouped_df['tn'] = scaler.fit_transform(grouped_df[['tn']])

# Definir parámetros de procesamiento en lotes
n_steps = 13
step_ahead = 2
batch_size = 10000

# Procesar datos en lotes y obtener modelos y predicciones
models, predictions = process_data_in_batches(grouped_df, clusters, n_steps, step_ahead, batch_size)

# Convertir las predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['cluster', 'prediction'])

# Aplicar los ratios para obtener las predicciones finales por product_id
final_predictions = []
for _, row in predictions_df.iterrows():
    cluster = row['cluster']
    cluster_indices = pivot_df[pivot_df['cluster'] == cluster].index
    for (cat1, cat2, cat3, brand, customer_id, product_id), ratio in ratio_dict.items():
        if any((pivot_df.loc[idx, ['cat1', 'cat2', 'cat3', 'brand', 'customer_id']] == [cat1, cat2, cat3, brand, customer_id]).all() for idx in cluster_indices):
            final_predictions.append([product_id, row['prediction'] * ratio])

final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediction'])

# Calcular el promedio de tn para los últimos 12 meses para cada product_id
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')
last_12_months = df[df['periodo'] >= df['periodo'].max() - pd.DateOffset(months=12)]
average_tn_last_12_months = last_12_months.groupby('product_id')['tn'].mean().reset_index()

# Identificar los product_id faltantes en final_predictions_df
missing_product_ids = set(df['product_id']) - set(final_predictions_df['product_id'])
average_predictions = average_tn_last_12_months[average_tn_last_12_months['product_id'].isin(missing_product_ids)]

# Concatenar predicciones finales y promedios
complete_predictions_df = pd.concat([final_predictions_df, average_predictions], ignore_index=True)

# Guardar las predicciones finales con el nombre de archivo original
complete_predictions_df.to_csv("C:/Users/Usuario/desktop/vero2/predicciones_finales_DTWv1.csv", index=False)


ValueError: pivot_df no contiene todas las columnas necesarias: ['cat1', 'cat2', 'cat3', 'brand', 'customer_id']